In [1]:
import pandas as pd

In [2]:
import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *

In [3]:
# import aggregated trips and reset index
trips_aggregated_df = pd.read_pickle(TRIPS_GROUPED_SPATIO_TEMPORAL_PATH).reset_index()
trips_aggregated_df.head(2)

,time_interval_start,start_hex_id,end_hex_id,demand
0,2019-01-20,881f1a1601fffff,881f1a1659fffff,3
1,2019-01-20,881f1a1609fffff,881f1a1643fffff,2


In [4]:
# add time features
trips_aggregated_df['hour'] = trips_aggregated_df.time_interval_start.dt.hour
trips_aggregated_df['day'] = trips_aggregated_df.time_interval_start.dt.day
trips_aggregated_df['weekday'] = trips_aggregated_df.time_interval_start.dt.weekday
trips_aggregated_df['month'] = trips_aggregated_df.time_interval_start.dt.month
# trips_aggregated_df['day_of_year'] = trips_aggregated_df.time_interval_start.dt.dayofyear
trips_aggregated_df = trips_aggregated_df.drop(columns=['time_interval_start'])
trips_aggregated_df.head(2)

,start_hex_id,end_hex_id,demand,hour,day,weekday,month
0,881f1a1601fffff,881f1a1659fffff,3,0,20,6,1
1,881f1a1609fffff,881f1a1643fffff,2,0,20,6,1


In [5]:
# add poi data on start and end hexagon of the trip
hexagons_with_pois_df = pd.read_pickle(HEXAGON_WITH_POIS_PATH)

trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_pois_df, left_on="start_hex_id", right_on="hex")
trips_aggregated_df = trips_aggregated_df.drop(columns={"hex", "hex_and_neighbors"})

# add '_start' suffix to poi columns
trips_aggregated_df = trips_aggregated_df.rename(
    columns={
        "sustenance_poi": "sustenance_poi_start",
        "public_transport_poi": "public_transport_poi_start",
        "education_poi": "education_poi_start",
        "arts_and_culture_poi": "arts_and_culture_poi_start",
        "sports_poi": "sports_poi_start",
    }
)

trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_pois_df, left_on="end_hex_id", right_on="hex")
trips_aggregated_df = trips_aggregated_df.drop(columns={"hex", "hex_and_neighbors"})

# add '_end' suffix to poi columns
trips_aggregated_df = trips_aggregated_df.rename(
    columns={
        "sustenance_poi": "sustenance_poi_end",
        "public_transport_poi": "public_transport_poi_end",
        "education_poi": "education_poi_end",
        "arts_and_culture_poi": "arts_and_culture_poi_end",
        "sports_poi": "sports_poi_end",
    }
)
trips_aggregated_df.head(2)

,start_hex_id,end_hex_id,demand,hour,day,weekday,month,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start,sustenance_poi_end,public_transport_poi_end,education_poi_end,arts_and_culture_poi_end,sports_poi_end
0,881f1a1601fffff,881f1a1659fffff,3,0,20,6,1,82,80,11,5,5,162,93,23,9,16
1,881f1a1601fffff,881f1a1659fffff,1,12,30,2,1,82,80,11,5,5,162,93,23,9,16


In [6]:
# encode start and end hexagons as one-hot vectors
start_hex_dummies = pd.get_dummies(trips_aggregated_df.start_hex_id).add_prefix('start_')
end_hex_dummies = pd.get_dummies(trips_aggregated_df.end_hex_id).add_prefix('end_')
trips_aggregated_df = pd.concat([trips_aggregated_df, start_hex_dummies, end_hex_dummies], axis=1)
trips_aggregated_df = trips_aggregated_df.drop(columns=['start_hex_id', 'end_hex_id'])
trips_aggregated_df.head(2)

,demand,hour,day,weekday,month,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start,...,end_881f1abb2dfffff,end_881f1abb31fffff,end_881f1abb35fffff,end_881f1abb39fffff,end_881f1abb61fffff,end_881f1abb63fffff,end_881f1abb65fffff,end_881f1abb67fffff,end_881f1abb69fffff,end_881f1abb6bfffff
0,3,0,20,6,1,82,80,11,5,5,...,0,0,0,0,0,0,0,0,0,0
1,1,12,30,2,1,82,80,11,5,5,...,0,0,0,0,0,0,0,0,0,0


In [7]:
trips_aggregated_df.to_pickle(MODEL_DATA_PATH)